In [2]:
### split the raw csv from download

In [24]:
import os
import pandas as pd

# Define file paths
input_folder = 'raw_data_from_download'
output_folder = 'processed_employment_data'
os.makedirs(output_folder, exist_ok=True)

# Employment strata
STRATA = ["full_time_employees", "part_time_employees", "employment"]

# Years to process
YEARS = range(2015, 2023)

# Define fixed column names
FIXED_COLUMNS = [
    "2011 super output area - lower layer", "mnemonic",
    "1 : Agriculture, forestry & fishing (A)", "2 : Mining, quarrying & utilities (B,D and E)",
    "3 : Manufacturing (C)", "4 : Construction (F)", "5 : Motor trades (Part G)",
    "6 : Wholesale (Part G)", "7 : Retail (Part G)", "8 : Transport & storage (inc postal) (H)",
    "9 : Accommodation & food services (I)", "10 : Information & communication (J)",
    "11 : Financial & insurance (K)", "12 : Property (L)",
    "13 : Professional, scientific & technical (M)",
    "14 : Business administration & support services (N)",
    "15 : Public administration & defence (O)", "16 : Education (P)", "17 : Health (Q)",
    "18 : Arts, entertainment, recreation & other services (R,S,T and U)"
]

skip_lines = [
    '"*","These figures',
    '"","The level of rounding',
    '"","further information',
    '"","https://www.nomisweb.co.uk',
    '"",""'
]

# Process each file
for year in YEARS:
    input_file = os.path.join(input_folder, f"{year}.csv")

    # Read the file
    with open(input_file, 'r') as infile:
        data = infile.readlines()

    # Initialize variables
    sections = []  # To store each strata section
    current_section = []  # To collect lines for the current strata

    # Process the file
    for line in data:
        if line.startswith('"Business Register'):  # Header line found
            if current_section:  # Save the previous section
                sections.append(current_section)
            current_section = [line]  # Start new section
        elif line.strip():  # Non-empty line, add to current section
            if not any(line.startswith(skip) for skip in skip_lines):
                current_section.append(line)

    # Append the last section if it exists
    if current_section:
        sections.append(current_section)

    # Check we have exactly 3 sections
    if len(sections) != 3:
        raise ValueError(f"Unexpected number of sections in file {input_file}. Expected 3, found {len(sections)}")

    # Write each section to its respective output file
    for strata_idx, strata in enumerate(STRATA):
        output_file = os.path.join(output_folder, f"employment_{year}_{strata}.csv")
        section = sections[strata_idx]

        # Locate the header and data lines
        for i, line in enumerate(section):
            if line.startswith('"2011 super output area'):
                header_index = i
                break
        data_lines = section[header_index:]   
        with open(output_file, 'w') as outfile:
            outfile.writelines(data_lines)  # Write data lines

        df = pd.read_csv(output_file)
        df = df.loc[:, ~df.columns.str.contains('Unnamed')]
        assert list(df.columns) == FIXED_COLUMNS, f"Column mismatch in {output_file}"
        df.to_csv(output_file, index=False)
        print(f"File saved: {output_file}")
    

File saved: processed_employment_data\employment_2015_full_time_employees.csv
File saved: processed_employment_data\employment_2015_part_time_employees.csv
File saved: processed_employment_data\employment_2015_employment.csv
File saved: processed_employment_data\employment_2016_full_time_employees.csv
File saved: processed_employment_data\employment_2016_part_time_employees.csv
File saved: processed_employment_data\employment_2016_employment.csv
File saved: processed_employment_data\employment_2017_full_time_employees.csv
File saved: processed_employment_data\employment_2017_part_time_employees.csv
File saved: processed_employment_data\employment_2017_employment.csv
File saved: processed_employment_data\employment_2018_full_time_employees.csv
File saved: processed_employment_data\employment_2018_part_time_employees.csv
File saved: processed_employment_data\employment_2018_employment.csv
File saved: processed_employment_data\employment_2019_full_time_employees.csv
File saved: processed_

In [25]:
import pandas as pd
data = pd.read_csv('processed_employment_data\employment_2022_employment.csv')

In [26]:
data.head()

,2011 super output area - lower layer,mnemonic,"1 : Agriculture, forestry & fishing (A)","2 : Mining, quarrying & utilities (B,D and E)",3 : Manufacturing (C),4 : Construction (F),5 : Motor trades (Part G),6 : Wholesale (Part G),7 : Retail (Part G),8 : Transport & storage (inc postal) (H),9 : Accommodation & food services (I),10 : Information & communication (J),11 : Financial & insurance (K),12 : Property (L),"13 : Professional, scientific & technical (M)",14 : Business administration & support services (N),15 : Public administration & defence (O),16 : Education (P),17 : Health (Q),"18 : Arts, entertainment, recreation & other services (R,S,T and U)"
0,City of London 001A,E01000001,0,175,20,500,0,300,75,125,400,1500,3500,800,4500,1250,150,175,1000,700
1,City of London 001B,E01000002,0,20,20,600,10,450,450,50,1000,5000,20000,1250,10000,4000,0,1000,1000,1250
2,City of London 001C,E01000003,0,0,10,10,0,5,40,0,50,100,50,50,100,300,10,10,15,35
3,City of London 001E,E01000005,10,150,50,900,0,225,175,450,1250,2500,7000,300,4500,4000,20,600,500,350
4,Barking and Dagenham 016A,E01000006,0,0,0,35,0,0,5,75,5,0,0,0,5,5,0,0,10,0


In [27]:
data.columns

Index(['2011 super output area - lower layer', 'mnemonic',
       '1 : Agriculture, forestry & fishing (A)',
       '2 : Mining, quarrying & utilities (B,D and E)',
       '3 : Manufacturing (C)', '4 : Construction (F)',
       '5 : Motor trades (Part G)', '6 : Wholesale (Part G)',
       '7 : Retail (Part G)', '8 : Transport & storage (inc postal) (H)',
       '9 : Accommodation & food services (I)',
       '10 : Information & communication (J)',
       '11 : Financial & insurance (K)', '12 : Property (L)',
       '13 : Professional, scientific & technical (M)',
       '14 : Business administration & support services (N)',
       '15 : Public administration & defence (O)', '16 : Education (P)',
       '17 : Health (Q)',
       '18 : Arts, entertainment, recreation & other services (R,S,T and U)'],
      dtype='object')